In [3]:
import argparse
import torch
from transformers import AutoModelForCausalLM, default_data_collator, AutoTokenizer
from peft import LoraConfig, get_peft_model
from accelerate import Accelerator
from accelerate.utils import GradScalerKwargs
import wandb
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
from typing import Optional

/home/vmasti/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Helper functions

def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params


def seed_all(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

## Get the training code

In [6]:
!git clone https://github.com/EfficientLLMs/lazy-pretrain.git

Cloning into 'lazy-pretrain'...
remote: Enumerating objects: 1035, done.
remote: Counting objects: 100% (493/493), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 1035 (delta 405), reused 381 (delta 299), pack-reused 542 (from 1)
Receiving objects: 100% (1035/1035), 35.89 MiB | 18.02 MiB/s, done.
Resolving deltas: 100% (785/785), done.


In [7]:
!git pull

remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 9 (delta 7), reused 9 (delta 7), pack-reused 0 (from 0)
Unpacking objects: 100% (9/9), 2.52 KiB | 859.00 KiB/s, done.
From https://github.com/EfficientLLMs/lazy-pretrain
   1e5fda4..1e3546d  main       -> origin/main
Updating 1e5fda4..1e3546d
Fast-forward
 run_scripts/pretrain_lora.sh  |  43 ++++++++++----
 run_scripts/run_grow.sh       |  28 +++++-----
 src/pretrain/pretrain_lora.py | 126 +++++++++++++++++++++++++++++++-----------
 src/pretrain/utils.py         |  25 +++++++--
 4 files changed, 157 insertions(+), 65 deletions(-)


In [1]:
%cd lazy-pretrain/

/home/vmasti/efficient-llms-capstone/lazy-pretrain


## Download and use the pre-tokenized data with the same order
EleutherAI has provided a pre-tokenized version of the standard (duplicated) pile dataset, which is also Pythia pre-shuffled. The dataset contains only token_ids. [link](https://huggingface.co/datasets/EleutherAI/pile-standard-pythia-preshuffled/tree/main)

The whole dataset has about 300B tokens. `00.bin` to `19.bin` are about 30GB large each. The last one `20.bin` is only 78.3MB. We can download only the last one.


1. Clone the repository without downloading


In [14]:
!mkdir data && cd data && GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datasets/EleutherAI/pile-standard-pythia-preshuffled

Cloning into 'pile-standard-pythia-preshuffled'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 30 (delta 1), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (30/30), 4.99 KiB | 852.00 KiB/s, done.


2. Download only the last file, which has 39_168_000 tokens


In [15]:
!cd data/pile-standard-pythia-preshuffled && git lfs pull --include="document-00020-of-00020.bin"

3. Get all the tokens from the last file. We can use it to get the last 5M tokens.

In [5]:
filename = "data/pile-standard-pythia-preshuffled/document-00020-of-00020.bin"
tokens = np.memmap(filename, dtype=np.uint16)
len(tokens)

39168000

In [13]:
!pip install ai2-olmo

## Grow the 400m model to 1.4b

In [6]:
!python src/grow/grow.py \
    --small_model "pythia-410m" \
    --large_depth 24 \
    --large_width 2048 \
    --depth_growth "alternate" \
    --attn_heads 16 \
    --output_dir "models/pythia-410m-to-pythia-1.4b"

Original model: 24 layers, 1024 width
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/vmasti/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Finish the following sentence:
Raindrops on roses,

A:

I think

Expanding model of 1024 width to 2048 width
attention ratio: 1.0
Grown model: 24 layers, 2048 width
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Finish the following sentence:
Raindrops on roses,

A:

I think

Grown model config: GPTNeoXConfig {
  "_name_or_path": "EleutherAI/pythia-410m-expand-width-2048",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_bias": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gel

# Pre-training a 1.4B model with limited compute using a 410m model

In [8]:
# Train

!python src/pretrain/pretrain_lora.py \
    --grown_model "models/pythia-410m-to-pythia-1.4b" \
    --tokenizer "EleutherAI/pythia-70m" \
    --seed 1234 \
    --rank 256 \
    --lora_alpha 256 \
    --batch_size 32 \
    --lr 1e-5 \
    --output_dir "models/pythia-410m-to-pythia-1.4b-lora-10m" \
    --dataset 'pile' \
    --num_tokens 10_000_000 \
    --chunk_size 1024 \
    --use_on_the_fly \
    --first_idx 19 \
    --last_idx 20 \
    --wandb_entity vibhamasti \
    --wandb_run_name "pythia-410-1.4b-10m-tokens"


device: cuda

Training configuration:
Total batches: 306
Number of GPUs: 1
Steps per GPU: 306
Total steps: 306
wandb: Currently logged in as: vibhamasti. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.19.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.5
wandb: Run data is saved locally in /home/vmasti/efficient-llms-capstone/lazy-pretrain/wandb/run-20250515_011333-ixiw0xmc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run pythia-410-1.4b-10m-tokens
wandb: ⭐️ View project at https://wandb.ai/vibhamasti/lora-pretraining
wandb: 🚀 View run at https://wandb.ai/vibhamasti/lora-pretraining/runs/ixiw0xmc/workspace
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:05<00:00,  2.61s/it]
Memory after enabling gradient checkpointing: 2794.25 MB
Model memory usage: 2794.25 MB
Max memory allocated: 2794.25 MB
Memory reserved: 2810.00 MB
Total trainable parameters: 50331648
Train